In [1]:
%load_ext autoreload

In [2]:
from wordle_game import wordle, solver, helper

In [3]:
import os
import numpy as np
from tqdm import tqdm
from typing import List

In [4]:
wordlist = wordle.WordList()
maxentropy = solver.MaxEntropy()
state_space = maxentropy.state_space

In [9]:
def guess(state_space, games: List[wordle.WordleGame]):
        """Determine optimal guess.
        """
        # TODO: Fix, have class that monitors number of guesses.
        if games[0].round == 1:
            return "soare"

        for game in games:
            if game.complete() != wordle.IN_PROGRESS:
                continue
            answers = solver.filter_answers(game)
            if len(answers) <= 2:
                return answers[0]
            elif len(answers) == 0:
                raise ValueError("No valid guesses left!")

        entropy = 0
        for game in games:
            if game.complete() != wordle.IN_PROGRESS:
                continue
            N, M = state_space.shape
            filter_mat = np.ones(N, dtype=int)
            for i, guess in enumerate(game.guesses):

                state = solver.convert_state(game.state[i])
                guess_num = wordlist.word_index(guess)
                filter_mat[state_space[:, guess_num] != state] = 0

            counts = helper.calculate_counts(state_space, filter_mat)

            entropy += helper.calc_entropy(counts)

        guess_num = np.argmin(entropy)
        guess = wordlist.acceptable_guesses[guess_num]

        return guess
    
def guess_word(word: str, games: List[wordle.WordleGame]):
    for game in games:
        if game.complete() != wordle.IN_PROGRESS:
                continue
        game.guess(word)
        
def print_games(games: List[wordle.WordleGame]):
    for i, game in enumerate(games):
        print("Game", i+1)
        print(game)
        
def complete(games):
    for game in games:
        if game.complete() == wordle.IN_PROGRESS:
            return wordle.IN_PROGRESS
   
    # TODO: Fix win/loss
    return wordle.WIN

def add_state(word, response, games):
    for game in games:
        if game.complete() != wordle.IN_PROGRESS:
                continue
        game.add_state(word, response)

In [6]:
def simulate(num_games, max_guesses=6):
    rounds = 1
    games = [wordle.WordleGame(wordlist=wordlist, max_guesses=max_guesses) for i in range(num_games)]
    while complete(games) == wordle.IN_PROGRESS:
        print("Round", rounds)
        rounds += 1
        word = guess(state_space, games)
        print("Guessing", word)
        guess_word(word, games)
        print_games(games)

In [7]:
simulate(2)

Round 1
Guessing soare
Game 1
SOARE

Game 2
SOARE

Round 2
Guessing clipt
Game 1
SOARE
CLIPT

Game 2
SOARE
CLIPT

Round 3
Guessing sheik
Game 1
SOARE
CLIPT
SHEIK

Game 2
SOARE
CLIPT
SHEIK

Round 4
Guessing sinew
Game 1
SOARE
CLIPT
SHEIK
SINEW

Game 2
SOARE
CLIPT
SHEIK
SINEW

Round 5
Guessing aduki
Game 1
SOARE
CLIPT
SHEIK
SINEW
ADUKI

Game 2
SOARE
CLIPT
SHEIK
SINEW

Round 6
Guessing chill
Game 1
SOARE
CLIPT
SHEIK
SINEW
ADUKI
CHILL

Game 2
SOARE
CLIPT
SHEIK
SINEW



In [8]:
simulate(4, max_guesses=9)

Round 1
Guessing soare
Game 1
SOARE

Game 2
SOARE

Game 3
SOARE

Game 4
SOARE

Round 2
Guessing prose
Game 1
SOARE
PROSE

Game 2
SOARE
PROSE

Game 3
SOARE
PROSE

Game 4
SOARE
PROSE

Round 3
Guessing cumin
Game 1
SOARE
PROSE
CUMIN

Game 2
SOARE
PROSE
CUMIN

Game 3
SOARE
PROSE
CUMIN

Game 4
SOARE
PROSE

Round 4
Guessing masse
Game 1
SOARE
PROSE
CUMIN
MASSE

Game 2
SOARE
PROSE
CUMIN
MASSE

Game 3
SOARE
PROSE
CUMIN
MASSE

Game 4
SOARE
PROSE

Round 5
Guessing tabid
Game 1
SOARE
PROSE
CUMIN
MASSE

Game 2
SOARE
PROSE
CUMIN
MASSE
TABID

Game 3
SOARE
PROSE
CUMIN
MASSE
TABID

Game 4
SOARE
PROSE

Round 6
Guessing broth
Game 1
SOARE
PROSE
CUMIN
MASSE

Game 2
SOARE
PROSE
CUMIN
MASSE
TABID
BROTH

Game 3
SOARE
PROSE
CUMIN
MASSE
TABID
BROTH

Game 4
SOARE
PROSE

Round 7
Guessing bribe
Game 1
SOARE
PROSE
CUMIN
MASSE

Game 2
SOARE
PROSE
CUMIN
MASSE
TABID
BROTH

Game 3
SOARE
PROSE
CUMIN
MASSE
TABID
BROTH
BRIBE

Game 4
SOARE
PROSE



In [23]:
def assistant(num_games, max_guesses=6):
    """Assist Wordle gameplay.
    """
    
    games = [wordle.WordleGame(wordlist=wordlist, max_guesses=max_guesses) for i in range(num_games)]

    while True:
        guess_word = input("Enter guess (enter empty string for no more guesses, list to see remaining valid answers):")
        if guess_word.strip() == "":
            best_guess = guess(state_space, games)
            print("Best guess is", best_guess)
            continue
        elif not wordlist.valid_guess(guess_word):
            print("Guess is not valid!")
            continue

        for i, game in enumerate(games):
            if game.complete() != wordle.IN_PROGRESS:
                continue
            response = input("Enter response for game #{}:".format(i+1))
            while not wordle.valid_state(response):
                print("Enter a valid response.")
                response = input("Enter response for game #{}:".format(i+1))
            game.add_state(guess_word, response)
            
        complete = game.complete()
        if complete == wordle.WIN:
            print("You won!")
            break
        elif complete == wordle.LOSE:
            print("You exceeded the maximum guesses and lost :(.")
            break

In [24]:
assistant(4, 9)

Enter guess (enter empty string for no more guesses, list to see remaining valid answers):soare
Enter response for game #1:BBBBY
Enter response for game #2:YBYBB
Enter response for game #3:BBGBY
Enter response for game #4:BBBBB
Enter guess (enter empty string for no more guesses, list to see remaining valid answers):
Best guess is linty
Enter guess (enter empty string for no more guesses, list to see remaining valid answers):linty
Enter response for game #1:GGYBB
Enter response for game #2:BBBGG
Enter response for game #3:BBBGB
Enter response for game #4:BBGBG
Enter guess (enter empty string for no more guesses, list to see remaining valid answers):
Best guess is liken
Enter guess (enter empty string for no more guesses, list to see remaining valid answers):liken
Enter response for game #1:GGGGG
Enter response for game #2:BBBBB
Enter response for game #3:BBBYB
Enter response for game #4:BBYBY
Enter guess (enter empty string for no more guesses, list to see remaining valid answers):
Bes